This script examines each period (from `INTERVAL_FILE`) and extract the income group designation of each country during the period (from `INCOME_GROUP`). It attaches the country name in WoS using `FLAG_TABLE` file and writes the resulting table to `CNTRY_YEAR_INCOMEGROUP` file. 

In [4]:
import sys
import numpy as np
import pandas as pd
from itertools import islice
from os.path import join as pjoin

# Parameters

This cell contains parameters that can be passed from `papermill`. 

In [14]:
WORK_DIR = "../../nsp/Data/Additional_data"
INCOME_GROUP = pjoin(WORK_DIR, "IncomeGroup.xlsx")
INTERVAL_FILE = pjoin(WORK_DIR, "year_intervals.txt")
FLAG_TABLE = pjoin(WORK_DIR, "country_WoS_ECI_WB_flags.tsv")
CNTRY_YEAR_INCOMEGROUP = pjoin("cntry_year_incomegroup.csv")

# Code

In [15]:
incomegroup_df = pd.read_excel(INCOME_GROUP, engine="openpyxl").replace(
    to_replace="..", value=np.nan
)
periods = [
    tuple(map(int, x.strip("\n").split("-")))
    for x in islice(open(INTERVAL_FILE), 3, None)
]

result_df = pd.DataFrame()
for (start, end) in periods:
    cols = list(np.arange(start, end + 1)) + ["Code"]
    subset_df = incomegroup_df[cols]
    mode_df = (
        subset_df.set_index("Code").mode(axis=1, dropna=True).reset_index()[["Code", 0]]
    )
    mode_df["YEAR"] = f"{start}-{end}"
    result_df = pd.concat([result_df, mode_df])
result_df.columns = ["Code", "IncomeGroup", "YEAR"]

cntry_flag = pd.read_csv(FLAG_TABLE, sep="\t")
result_df = result_df.merge(cntry_flag[["WoS", "Code"]], on="Code")

result_df.to_csv(CNTRY_YEAR_INCOMEGROUP, index=False)


# Explanation

In [7]:
incomegroup_df = pd.read_excel(INCOME_GROUP, engine='openpyxl').replace(to_replace="..", value=np.nan)
incomegroup_df

,Code,Data for calendar year :,1987,1988,1989,1990,1991,1992,1993,1994,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,AFG,Afghanistan,L,L,L,L,L,L,L,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALB,Albania,NaN,NaN,NaN,LM,LM,LM,L,L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DZA,Algeria,UM,UM,LM,LM,LM,LM,LM,LM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ASM,American Samoa,H,H,H,UM,UM,UM,UM,UM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AND,Andorra,NaN,NaN,NaN,H,H,H,H,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,MYT,Mayotte,NaN,NaN,NaN,H,UM,UM,UM,UM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220,ANT,Netherlands Antilles (former),UM,UM,UM,UM,UM,UM,UM,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221,YUG,Serbia and Montenegro (former),NaN,NaN,NaN,NaN,NaN,LM,LM,LM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
222,SUN,USSR (former),NaN,NaN,NaN,UM,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
open(INTERVAL_FILE).readlines()[3:]

['1988-1992\n',
 '1993-1997\n',
 '1998-2002\n',
 '2003-2007\n',
 '2008-2012\n',
 '2013-2017']

In [9]:
periods = [tuple(map(int, x.strip("\n").split('-'))) for x in islice(open(INTERVAL_FILE), 3, None)]
periods

[(1988, 1992),
 (1993, 1997),
 (1998, 2002),
 (2003, 2007),
 (2008, 2012),
 (2013, 2017)]

In [10]:
cols = list(np.arange(1988, 1992)) + ["Code"]
cols

[1988, 1989, 1990, 1991, 'Code']

In [11]:
subset_df = incomegroup_df[cols]
subset_df.head()

,1988,1989,1990,1991,Code
0,L,L,L,L,AFG
1,NaN,NaN,LM,LM,ALB
2,UM,LM,LM,LM,DZA
3,H,H,UM,UM,ASM
4,NaN,NaN,H,H,AND


In [12]:
mode_df = subset_df.set_index("Code").mode(axis=1, dropna=True)
mode_df["YEAR"] = "1988-1992"
mode_df.head()

,0,1,YEAR
Code,,,
AFG,L,NaN,1988-1992
ALB,LM,NaN,1988-1992
DZA,LM,NaN,1988-1992
ASM,H,UM,1988-1992
AND,H,NaN,1988-1992


In [13]:
cntry_flag = pd.read_csv(FLAG_TABLE, sep="\t")
cntry_flag.head()


,WoS,ECI_Country,WB,Code
0,Albania,Albania,Albania,ALB
1,Algeria,Algeria,Algeria,DZA
2,Angola,Angola,Angola,AGO
3,Argentina,Argentina,Argentina,ARG
4,Australia,Australia,Australia,AUS
